<header>
   <p  style='font-size:36px;font-family:Arial; color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       GLM Fraud Detection with Python and Vantage BYOM
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Introduction</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>
    In recent years we have seen a massive increase in Fraud attempts, making fraud detection necessary for Banking and Financial Institutions. Despite countless efforts and human supervision, hundreds of millions are lost due to fraud. Fraud can happen using various methods, i.e., stolen credit cards, misleading accounting, phishing emails, etc. Due to small cases in significant populations, the detection of fraud is essential as well as challenging.
    <br>
    <br>
    This notebook provides a demonstration of "data science workflow" that illustrates how to leverage Vantage's <b>Bring Your Own Model</b> to build a model in python and score the model at scale in Vantage. Users perform large-scale operations such as feature analysis, data transformation, and ML Model Scoring in the Vantage environment without moving data.
    <br>
    <br>
    Data Scientists can use their preferred language, tools and platform to develop models. We can then score these models in Vantage without moving the data and hence giving an advantage in the performance and scalability of Vantage. This shortens the time to production.
    
</p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Steps in the analysis:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Connect to Vantage</li>
    <li>Read the data from Vantage as a teradaml Dataframe</li>
    <li>Clean up the dataset</li>
    <li>Create training and testing datasets in Vantage</li>
    <li>Create a PMML pipeline locally using sklearn and train it</li>
    <li>Load trained PMML Model into Vantage</li>
    <li>Score the model directly in Vantage</li>
    <li>Visualize the results (ROC curve and AUC)</li>
    <li>Cleanup</li>
</ol>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Installing some dependencies</b>

In [ ]:
%%capture
# # '%%capture' suppresses the display of installation steps of the following packages
# !pip install sklearn2pmml --user
# !pip install jdk4py --user

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we import the required libraries, set environment variables and environment paths (if required).</p>

In [ ]:
# import statements
import os
import getpass
import warnings
import pandas as pd
import matplotlib.pyplot as plt
from jdk4py import JAVA, JAVA_HOME, JAVA_VERSION

from teradataml import *
from teradataml import PMMLPredict
from teradataml.dataframe.dataframe import DataFrame

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml

warnings.filterwarnings("ignore")
%matplotlib inline
configure.byom_install_location = 'mldb'
display.max_rows = 5

# Set java path

os.environ['PATH'] = os.environ['PATH'] + os.pathsep + str(JAVA_HOME)
os.environ['PATH'] = os.environ['PATH'] + os.pathsep + str(JAVA)[:-5]

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>1. Connect to Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>You will be prompted to provide the password. Enter your password, press the Enter key, then use down arrow to go to next cell.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql("SET query_band='DEMO=GLM_Fraud_Detection_BYOM.ipynb;' UPDATE FOR SESSION;")

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Begin running steps with Shift + Enter keys. </p>

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. You can either run the demo using foreign tables to access the data without any storage on your environment or download the data to local storage, which may yield faster execution. Still, there could be considerations of available storage. Two statements are in the following cell, and one is commented out. You may switch which mode you choose by changing the comment string.</p>

In [ ]:
%run -i ../run_procedure.py "call get_data('demo_glm_fraud_cloud');"        # Takes 1 minute
# %run -i ../run_procedure.py "call get_data('demo_glm_fraud_local');"        # Takes 2 minutes

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next is an optional step – if you want to see status of databases/tables created and space used. </p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"        # Takes 10 seconds

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>2. Read the data from Vantage as a teradaml Dataframe</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The data from <a href = 'https://www.kaggle.com/code/georgepothur/4-financial-fraud-detection-xgboost/data'>https://www.kaggle.com/code/georgepothur/4-financial-fraud-detection-xgboost/data</a> is loaded in Vantage in a table named "transaction_data". Check the data size and print sample rows: 63k rows and 12 columns.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Please scroll down to the end of the notebook for detailed column descriptions of the dataset.</p>

In [ ]:
# Get data as a teradataml dataframe
txn_data = DataFrame(in_schema('DEMO_GLM_Fraud','transaction_data'))

print("Shape of the data: ", txn_data.shape)
txn_data

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here we rename a misspelt column without moving the data out of Vantage. We are renaming <b>oldbalanceOrg</b> to <b>oldbalanceOrig</b></p>

In [ ]:
new_data = txn_data.assign(oldbalanceOrig = txn_data.oldbalanceOrg).drop(['oldbalanceOrg'], axis=1)

new_data

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Fraudulent agents inside a simulation make these transactions. In this specific dataset, the fraudulent behavior of the agents aims to profit by taking control or customers' accounts and trying to empty the funds by transferring them to another account and then cashing out of the system.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Below are some insights about the dataset:</b></p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>There are 92 fraud transactions i.e. 0.14% of fraud transactions in the dataset.</li>
    <li>From these 92 fraud transactions, 47 are of type TRANSFER and 45 are of type CASH_OUT.</li>
    <li>97.83% of fraud transactions have transaction amount equal to oldbalanceOrig i.e. account cleanout.</li>
    <li>71.74% of fraud transactions have recipient's old balance as zero.</li>
    <li>isFlaggedFraud is correct only two times among the 92 fraud transactions.</li>
</ol>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>3. Clean up the dataset</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Based on what we discovered above, we will:</p>
<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li>Remove all data that isn't 'CASH OUT' or 'TRANSFER'.</li>
    <li>Drop "nameOrig" and "nameDest" since the origin and destination accounts don't matter.</li>
    <li>Drop "isFlaggedFraud" because it has just flagged two transactions. Hence it doesn't have much significance.</li>
</ol> 

In [ ]:
clean_data = new_data.loc[(new_data.type == 'CASH_OUT') | (new_data.type == 'TRANSFER')]
clean_data.shape

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now our dataset is reduced to 27k records.</p>

In [ ]:
clean_data = clean_data.drop(['nameDest', 'nameOrig', 'isFlaggedFraud'], axis = 1)
clean_data

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here we'll copy this Teradata Dataframe to a separate table in Vantage.</p>

In [ ]:
# Create the source data table in the database
clean_data.to_sql('clean_data', if_exists = 'replace', primary_index = 'txn_id')

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>4. Create training and testing datasets in Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We'll create two datasets for training and testing in the ratio of 80:20.</p>

In [ ]:
# Create TRAIN table using SAMPLE

qry = '''
CREATE MULTISET TABLE clean_data_train 
  AS(SELECT * FROM clean_data SAMPLE 0.8) 
WITH DATA
PRIMARY INDEX (txn_id);
'''

try:
    execute_sql(qry)
except:
    db_drop_table('clean_data_train')
    execute_sql(qry)

In [ ]:
# Create TEST table using SAMPLE

qry = '''
CREATE MULTISET TABLE clean_data_test
  AS(SELECT * FROM clean_data
    EXCEPT
    SELECT * FROM clean_data_train)
WITH DATA
PRIMARY INDEX (txn_id); 
'''

try:
    execute_sql(qry)
except:
    db_drop_table('clean_data_test')
    execute_sql(qry)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>5. Create a PMML pipeline locally using sklearn and train it</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here we will use Logistic Regression as a classifier model in our PMML pipeline. The Predictive Model Markup Language (PMML) is an XML-based language which provides a way for applications to define machine learning, statistical and data mining models and to share models between PMML-compliant applications.</p>

In [ ]:
# creating TD dataframes for training and testing datasets
clean_data_train = DataFrame('clean_data_train')
clean_data_test = DataFrame('clean_data_test')

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 Create a local pandas dataframe.</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This dataframe is created outside Vantage. The model is trained outside Vantage. The model would be later saved to Vantage, and in-database scoring would be performed at scale.</p>

In [ ]:
clean_data_train_df = clean_data_train.to_pandas(all_rows = True)
clean_data_train_df.reset_index(inplace = True)

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 Train the Pipeline and save the model locally</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The model would be saved in a file called <b>mm_fraud_glm_model.pmml</b></p>

In [ ]:
warnings.filterwarnings("ignore")

y_train = clean_data_train_df[['isFraud']].astype(int)
X_train = clean_data_train_df[['step','amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig']].astype(float)


pipeline = PMMLPipeline([
     ("classifier", LogisticRegression())
     ])
pipeline.fit(X_train, y_train)

sklearn2pmml(pipeline, "mm_fraud_glm_model.pmml", with_repr = True)

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>6. Load trained PMML Model into Vantage</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Pass the local PMML file to the save_byom function. This function would load the <b>mm_fraud_glm_model.pmml</b> file into Vantage in a table called <b>mm_glm</b>. If there is already a model with the same name, we will delete the previous one, and a new one will be loaded.</p>

In [ ]:
# Load the PMML file into Vantage
try:
    res = save_byom(model_id = 'mm_glm1', model_file = 'mm_fraud_glm_model.pmml', table_name = 'mm_glm')

except Exception as e:
    # if our model exists, delete and rewrite
    if str(e.args).find('TDML_2200') >= 1:
        res = delete_byom(model_id = 'mm_glm1', table_name = 'mm_glm')
        res = save_byom(model_id = 'mm_glm1', model_file = 'mm_fraud_glm_model.pmml', table_name = 'mm_glm')
        pass
    else:
        raise

# Obtain a pointer to the model
model_tdf = DataFrame.from_query("SELECT * FROM mm_glm WHERE model_id = 'mm_glm1'")

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>7. Score the model directly in Vantage</b>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>PMMLPredict will score the model on data resident in Vantage without moving data.</p>

In [ ]:
# Run the PMMLPredict function in Vantage
result = PMMLPredict(
            modeldata = model_tdf,
            newdata = clean_data_test,
            accumulate = ['step','amount','newbalanceOrig','oldbalanceDest','newbalanceDest','oldbalanceOrig', 'isFraud'],
            overwrite_cached_models = '*',
            model_output_fields = ['probability(1)', 'probability(0)']
            )

In [ ]:
# Create a local pandas dataframe of the results
result_df = result.result.to_pandas(all_rows = True)
result_df.head()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The output above shows probability(1), i.e. transaction being fraud and probability(0), i.e. transaction being genuine.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Value in the prediction field of the result column in json_report appears only if the result column in json_report has a prediction field. The prediction output column could be empty if the scoring performed with a classification model does not return a predicted value.
    <br>
    <br>
Read more about it <a href = 'https://docs.teradata.com/search/all?query=PMMLPredict+Output&content-lang=en-US'>here</a></p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>8. Visualize the results (ROC curve and AUC)</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Calculate mean absolute error and AUC(Area Under the Curve) for Receiver Operating Characteristic Curve</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Mean Absolute Error is the summation of the difference between actual and predicted values averaged over the number of observations.</p>

In [ ]:
print(mean_absolute_error(result_df['isFraud'], result_df['probability(1)']))

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ROC curve is a graph between TPR(True Positive Rate) and FPR(False Positive Rate). The area under the ROC curve is a metric of how well the model can distinguish between positive and negative classes. The higher the AUC, the better the model's performance in distinguishing between the positive and negative categories. AUC above 0.75 is generally considered decent.</p>

In [ ]:
AUC = roc_auc_score(result_df['isFraud'], result_df['probability(1)'])
AUC

In [ ]:
fpr, tpr, thresholds = roc_curve(result_df['isFraud'], result_df['probability(1)'])
plt.plot(fpr, tpr, color = 'orange', label = 'ROC. AUC = {}'.format(str(AUC)))
plt.plot([0, 1], [0, 1], color = 'darkblue', linestyle = '--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curve')
plt.legend()
plt.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Looking at the above ROC Curve, we can confidently say that the model has performed well on testing data. The AUC value is way above 0.75 and resonates with our understanding that the model is performing well.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>9. Cleanup</b>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Work Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Cleanup work tables to prevent errors next time.</p>

In [ ]:
tables = ['clean_data_train', 'clean_data_test', 'mm_glm']

# Loop through the list of tables and execute the drop table command for each table

for table in tables:
    try:
        db_drop_table(table_name = table)
    except:
        pass

<p style = 'font-size:18px;font-family:Arial;color:#00233C'> <b>Databases and Tables </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('demo_glm_fraud');"        # Takes 5 seconds

In [ ]:
remove_context()

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Dataset:</b>

- `txn_id`: transaction id
- `step`: maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (31 days simulation).
- `type`: CASH-IN, CASH-OUT, DEBIT, PAYMENT and TRANSFER
- `amount`: amount of the transaction in local currency
- `nameOrig`: customer who started the transaction
- `oldbalanceOrig`: customer's balance before the transaction
- `newbalanceOrig`: customer's balance after the transaction
- `nameDest`: customer who is the recipient of the transaction
- `oldbalanceDest`: recipient's balance before the transaction
- `newbalanceDest`: recipient's balance after the transaction
- `isFraud`: identifies a fraudulent transaction (1) and non fraudulent (0)
- `isFlaggedFraud`: flags illegal attempts to transfer more than 200,000 in a single transaction

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Links:</b></p>
<ul style = 'font-size:16px;font-family:Arial'>
    <li>Uses a dataset and feature discovery methods outlined here: <a href = 'https://www.kaggle.com/georgepothur/4-financial-fraud-detection-xgboost/notebook'>https://www.kaggle.com/georgepothur/4-financial-fraud-detection-xgboost/notebook</a></li>
    <li>Teradataml Python reference: <a href = 'https://docs.teradata.com/search/all?query=Python+Package+User+Guide&content-lang=en-US'>here</a></li>
</ul>

<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            Copyright © Teradata Corporation - 2023. All Rights Reserved
        </div>
    </div>
</footer>